In [ ]:
import nshutils as nu

nu.pretty()

In [2]:
from __future__ import annotations

import os
from pathlib import Path

import mattertune.configs as MC
from mattertune import MatterTuner

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

def hparams():
    hparams = MC.MatterTunerConfig.draft()

    # Model hparams
    hparams.model = MC.M3GNetBackboneConfig.draft()
    hparams.model.ckpt_path = "M3GNet-MP-2021.2.8-PES"
    hparams.model.graph_computer = MC.M3GNetGraphComputerConfig.draft()
    hparams.model.optimizer = MC.AdamWConfig(lr=1.0e-4)
    
    ## Properties
    hparams.model.properties = []
    energy = MC.EnergyPropertyConfig(loss=MC.MAELossConfig())
    hparams.model.properties.append(energy)
    forces = MC.ForcesPropertyConfig(loss=MC.MAELossConfig(), conservative=True)
    hparams.model.properties.append(forces)
    
    ## Data hparams
    hparams.data = MC.AutoSplitDataModuleConfig.draft()
    hparams.data.dataset = MC.XYZDatasetConfig.draft()
    hparams.data.dataset.src = "./data/water_ef.xyz"
    hparams.data.train_split = 0.8
    hparams.data.batch_size = 1
    
    # Trainer hparams
    hparams.trainer.additional_trainer_kwargs = {"fast_dev_run": True}
    
    hparams = hparams.finalize(strict=False)
    return hparams
    
model = MatterTuner(hparams()).tune()
model

# pip install torch_sparse -f https://data.pyg.org/whl/torch-2.2.1+cu121.html

Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name          | Type            | Params | Mode 
-------------------------------

Epoch 0: 100%|██████████| 1/1 [00:07<00:00,  0.13it/s]

/net/csefiles/coc-fung-cluster/lingyu/miniconda3/envs/matgl-tune/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:07<00:00,  0.13it/s]


TuneOutput(model=M3GNetBackboneModule(
  (backbone): M3GNet(
    (bond_expansion): BondExpansion(
      (rbf): SphericalBesselFunction()
    )
    (embedding): EmbeddingBlock(
      (activation): SiLU()
      (layer_node_embedding): Embedding(89, 64)
      (layer_edge_embedding): MLP(9 → 64, SiLU)
    )
    (basis_expansion): SphericalBesselWithHarmonics(
      (shf): SphericalHarmonicsFunction()
      (sbf): SphericalBesselFunction()
    )
    (three_body_interactions): ModuleList(
      (0-2): 3 x ThreeBodyInteractions(
        (update_network_atom): MLP(64 → 9, Sigmoid)
        (update_network_bond): GatedMLP(
          (layers): Sequential(
            (0): Linear(in_features=9, out_features=64, bias=False)
            (1): SiLU()
          )
          (gates): Sequential(
            (0): Linear(in_features=9, out_features=64, bias=False)
            (1): Sigmoid()
          )
        )
      )
    )
    (graph_layers): ModuleList(
      (0-2): 3 x M3GNetBlock(
        (activation